In [1]:
%matplotlib
%matplotlib notebook

# Import Libraries
import cv2
import numpy as np
import pandas as pd
import os
import struct
import matplotlib.pyplot as plt

# Headers for different tables
meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']

distance_header = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan', 'LeftBolwigOrgan', 'RightBolwigOrgan']

def readSplineData(fileName, nFrames):
    fCount = 0;
    spline = {}
    with open(fileName, "rb") as f:
        while (True) and (fCount < nFrames-1):
            fCount += 1
            gap, frameNumber = struct.unpack('>ii', f.read(struct.calcsize('>ii')))

            nPointsToRead =  struct.unpack('>i', f.read(struct.calcsize('>i')))
            fmt = ">%dH" % (nPointsToRead)
            tempX = struct.unpack(fmt, f.read(struct.calcsize(fmt)))

            nPointsToRead =  struct.unpack('>i', f.read(struct.calcsize('>i')))
            fmt = ">%dH" % (nPointsToRead)
            tempY = struct.unpack(fmt, f.read(struct.calcsize(fmt)))
            
            spline[frameNumber-1] = np.vstack((np.asarray(tempX).T, np.asarray(tempY).T))

    return spline

def readContourData(fileName, nFrames):    
    fCount = 0;
    contour = {}
    with open(fileName, "rb") as f:
        while (True) and (fCount < nFrames-1):
            fCount += 1
            frameNumber = struct.unpack('>i', f.read(struct.calcsize('>i')))

            nPointsToRead = struct.unpack('>i', f.read(struct.calcsize('>i')))            
            fmt = ">%dH" %(nPointsToRead)
            buff = f.read(struct.calcsize(fmt))
            tempX = struct.unpack(fmt, buff)
            
            nPointsToRead = struct.unpack('>i', f.read(struct.calcsize('>i')))
            fmt = ">%dH" %(nPointsToRead)
            buff = f.read(struct.calcsize(fmt))
            tempY = struct.unpack(fmt, buff)
            
            frameNumber = frameNumber[0]
            contour[frameNumber-1] = np.vstack((np.asarray(tempX).T, np.asarray(tempY).T))

    return contour


Using matplotlib backend: MacOSX


In [2]:
# test_dir = root + 'dataCollectedOn_20180417_grp_2/Rawdata_20180417_083618_20180420_104633/'
# test_string = "Rawdata_20180417_083618" ## Start frame in zero index

# test_dir = root + 'dataCollectedOn_20180417_grp_3/Rawdata_20180417_082627_20180525_112343_A_133410/'
# test_string = "Rawdata_20180417_082627" ## Start frame in zero index

# test_dir = root + 'dataCollectedOn_20180417_grp_4/Rawdata_20180417_075246_20180420_092232_A_095851/'
# test_string = "Rawdata_20180417_075246" ## Start frame in zero index

# test_dir = root + 'dataCollectedOn_20180417_grp_5/Rawdata_20180417_072307_20180523_100127_A_122116/'
# test_string = "Rawdata_20180417_072307" ## Start frame in zero index

# test_dir = root + 'dataCollectedOn_20180417_grp_6/Rawdata_20180417_070739_20180522_151012/'
# test_string = "Rawdata_20180417_070739" ## Start frame in zero index

# test_dir = root + 'dataCollectedOn_20180417_grp_7/Rawdata_20180417_065725_20180522_114055/'
# test_string = "Rawdata_20180417_065725" ## Start frame in zero index

root = '../expts/'

all_exp = ['20180417_Individual', '20180417_7c1_Hess_100',
           '20180417_7c1_Hess_50', '20180417_7c1_Hess_25',
           '20180417_7c1_Nbr_25', '20180417_7c1_Hess_50_BothDO',
           '20180417_7c1_Hess_50_LeftDO', '20170317_7c0']

test = 'dataCollectedOn_20180417_grp_1/Rawdata_20180417_084502_20180423_075225/'
test_string = "Rawdata_20180417_084502" ## Start frame in zero index

# test = 'dataCollectedOn_20180417_grp_2/Rawdata_20180417_083618_20180420_104633/'
# test_string = "Rawdata_20180417_083618" ## Start frame in zero index

bp='LeftDorsalOrgan'
fig = plt.figure(figsize=(18, 9))
# for exp in [all_exp[i] for i in (0, 2)]:
for idx, exp in enumerate(all_exp):    
    test_dir = os.path.join(root, 'trainingData_'+ exp, test)

    ## Read tracker metadata and annotation
    for fs in os.listdir(test_dir):
        if 'Metadata' in fs:
            meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
        if 'Coordinates' in fs:
            coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)
        
    if (meta_data.empty is False):
        ## Make the metadata frame umber start from 0 index
        meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'] - 1
        meta_data.set_index('FrameNumber', inplace=True)

        ## Make the annotation frame umber start from 0 index
        coordinates = coordinates.round(0)
        start_frame = coordinates.loc[0, 'FrameNumber'].copy() - 1
        coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'] - start_frame
        coordinates.set_index('FrameNumber', inplace=True)
        no_anno = np.array(np.where(coordinates.values == -1)).T
        temp_index = coordinates.iloc[no_anno[:, 0], :].index.values
        temp_cols = coordinates.iloc[:,no_anno[:, 1]].columns.values
        no_anno_cols = []
        no_anno_index = []        
        for i in range(0, len(temp_cols), 2):
            no_anno_cols.append(temp_cols[i][:-2])
            no_anno_index.append(temp_index[i])
        no_anno_df = pd.DataFrame({'index': no_anno_index, 'col': no_anno_cols})
        
        ## Inner join to take only the frames intersecting the annotation and tracker metadata
        ## Use when want to see only annotated frames 
        meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='inner', suffixes=('_T', '_A'))

        ## Outer join to take only the frames intersecting the annotation and tracker metadata
        ## Use when want to see all frames 
#         meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='outer', suffixes=('_T', '_A'))

        ## Calculate distance between annotation and the tracker metadata
        distance = pd.DataFrame([], index=meta_coord.index.values)
        distance.index.name = 'FrameNumber'
        for head in distance_header:
            temp_x = (meta_coord[head+'_x_T'].values - meta_coord[head+'_x_A'].values)**2
            temp_y = (meta_coord[head+'_y_T'].values - meta_coord[head+'_y_A'].values)**2
            distance.loc[:, head+'_dist'] = np.sqrt(temp_x + temp_y)
        distance[distance > 2716] = np.nan
        distance = distance.copy()*2.75

        ## Now append the distance table to the main table 'meta_coord'
        meta_coord = meta_coord.merge(distance, on='FrameNumber')
        
        ## Replace the no annotation with NA
        for rind, rval in no_anno_df.iterrows():
            meta_coord.loc[rval['index'], rval['col']+'_dist'] = np.nan
            
        ## Replace NaN values with a high negative number
#         meta_coord.fillna(-2, inplace=True)
        meta_coord.dropna(axis=0, how='any')
        
        temp = None
        if idx == 0:
            dist_all_exp = np.array(meta_coord.index.values, dtype=np.int32)
        
        temp = np.array(meta_coord.loc[:, bp+'_dist'].values, dtype=np.float32)
        dist_all_exp = np.column_stack((dist_all_exp, temp))
        
        ## Plot Temporal
        plt.plot(meta_coord.index.values, meta_coord.loc[:, 'MouthHook_dist'].values, label=exp)
        plt.ylim([0, 250])

plt.legend(loc = 'upper right', frameon=False, fontsize=14)
plt.show()
plt.close('all')

In [3]:
all_exp_names = ['Same', 'Hess 100',
                 'Hess 50', 'Hess 25',
                 'Hess 100 Nbr 25', 'Hess 50 BothDO',
                 'Hess 50 LeftDO', '2017']
df = pd.DataFrame(dist_all_exp, columns=['index'] + all_exp_names)
df.set_index('index', inplace=True)

In [4]:
df.head()

,Same,Hess 100,Hess 50,Hess 25,Hess 100 Nbr 25,Hess 50 BothDO,Hess 50 LeftDO,2017
index,,,,,,,,
1.0,4763.172852,4763.172852,4763.172852,4763.172852,4763.172852,4763.172852,4763.172852,4763.172852
2.0,2.750000,43.044308,229.241821,18.447561,13.750000,14.809203,137.829605,38.890873
3.0,3.889087,12.298374,8.696263,28.044607,3.889087,200.900635,13.750000,67.136055
4.0,2.750000,8.696263,30.745935,8.696263,270.634094,210.100723,25.943449,165.571930
5.0,2.750000,6.149187,25.353746,19.445436,301.410553,110.068726,222.733017,165.091644


In [5]:
import seaborn as sns;
df[(df > 0) & (df <= 25)] = 25
df[(df > 25) & (df <= 50)] = 50
df[(df > 50) & (df <= 75)] = 75
df[(df > 75) & (df <= 100)] = 100
df[(df > 100) & (df <= 125)] = 125
df[df > 125] = 150
df = df.sort_values(by='Same')

In [6]:
# g = sns.clustermap(df, yticklabels=False,  cmap=sns.light_palette("navy", reverse=True))
# g = sns.clustermap(df, yticklabels=False,  cmap=sns.light_palette((210, 90, 60), input="husl"))


# fig = plt.figure(figsize=(2, 10))
g = sns.heatmap(df, yticklabels=False,
                cmap=sns.light_palette('green'),
                cbar=False)
g.set_xticks(g.get_xticks()+0.5)
g.set_xticklabels(g.get_xticklabels(), rotation=45, ha="right")
plt.tick_params(axis='x', bottom=False, top=False)
plt.ylabel('')
plt.savefig('test.png', bbox_inches='tight')
# plt.colorbar.set_ticks([0, 25, 50, 75, 100])
# cax = plt.gcf().axes[-1]
# cax.set_yticks([0, 25, 50, 75, 100, 125])
# cax.tick_params(labelsize=20)

In [7]:
# g = sns.clustermap(df, yticklabels=False,  cmap=sns.light_palette('green'), robust=True, figsize=(3, 9), col_cluster=True, row_cluster=False)

In [8]:
root = '../expts/'

all_exp = ['20180417_Individual', '20180417_7c1_Hess_100',
           '20180417_7c1_Hess_50', '20180417_7c1_Hess_25',
           '20180417_7c1_Nbr_25', '20180417_7c1_Hess_50_BothDO',
           '20180417_7c1_Hess_50_LeftDO', '20170317_7c0']

test = 'dataCollectedOn_20180417_grp_1/Rawdata_20180417_084502_20180423_075225/'
test_string = "Rawdata_20180417_084502" ## Start frame in zero index

# test = 'dataCollectedOn_20180417_grp_2/Rawdata_20180417_083618_20180420_104633/'
# test_string = "Rawdata_20180417_083618" ## Start frame in zero index

bp='LeftDorsalOrgan'
fig = plt.figure(figsize=(18, 9))
# for exp in [all_exp[i] for i in (0, 2)]:
for idx, exp in enumerate(all_exp):    
    test_dir = os.path.join(root, 'trainingData_'+ exp, test)

    ## Read tracker metadata and annotation
    for fs in os.listdir(test_dir):
        if 'Metadata' in fs:
            meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
        if 'Coordinates' in fs:
            coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)
        
    if (meta_data.empty is False):
        ## Make the metadata frame umber start from 0 index
        meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'] - 1
        meta_data.set_index('FrameNumber', inplace=True)

        ## Make the annotation frame umber start from 0 index
        coordinates = coordinates.round(0)
        start_frame = coordinates.loc[0, 'FrameNumber'].copy() - 1
        coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'] - start_frame
        coordinates.set_index('FrameNumber', inplace=True)